In [1]:
import datetime
import algorithms
import data_clean
import pandas as pd
import numpy as np

In [2]:
df = data_clean.do(data_clean)
df = algorithms.ml_probabilities(algorithms,df)
df_historic_win_data = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "1990-9-10") #winners by spread since
win_dict = algorithms.win_percent(df_historic_win_data)



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [39]:
#simulate rough draft
ev_dates = ev_games[(ev_games['schedule_date'] > '2013-09-09') & (ev_games['schedule_date'] < '2014-03-09')]
ev_dates = ev_dates.sort_values(by = 'schedule_date')
ev_dates


,Bet_Type,EV,away,away_ML,home,home_ML,implied_win_away,implied_win_home,schedule_date,score_away,score_home,spread_favorite,team_favorite_id,total_implied_prob
1094,LVR,0.382371,JAX,175.0,LVR,-202.5,0.363636,0.669421,2013-09-15,9.0,19.0,-5.0,LVR,1.033057
1086,BAL,0.517764,CLE,220.0,BAL,-250.0,0.312500,0.714286,2013-09-15,6.0,14.0,-7.0,BAL,1.026786
1089,GB,0.576367,WAS,310.0,GB,-362.5,0.243902,0.783784,2013-09-15,20.0,38.0,-8.0,GB,1.027686
1088,CHI,0.463164,MIN,227.5,CHI,-262.5,0.305344,0.724138,2013-09-15,30.0,31.0,-6.5,CHI,1.029482
1095,PHI,0.634456,LAC,270.0,PHI,-322.5,0.270270,0.763314,2013-09-15,33.0,30.0,-7.5,PHI,1.033584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326,SF,0.108834,SF,-174.0,GB,153.0,0.635036,0.395257,2014-01-05,23.0,20.0,-3.0,SF,1.030293
1328,SEA,0.661177,NO,332.5,SEA,-395.0,0.231214,0.797980,2014-01-11,15.0,23.0,-9.0,SEA,1.029194
1330,DEN,0.645549,LAC,289.0,DEN,-345.0,0.257069,0.775281,2014-01-12,17.0,24.0,-7.5,DEN,1.032350
1332,SEA,0.267492,SF,175.0,SEA,-196.5,0.363636,0.662732,2014-01-19,17.0,23.0,-4.0,SEA,1.026368


In [40]:
#https://www.investopedia.com/terms/w/win-loss-ratio.asp
#w_l is a trader's number of winning trades relative to the number of osing trades.

#A negative Kelly criterion means that the bet is not favored by the model and should be avoided.

def kelly_criterion(win_dict, fav_spread, bet_type, team_favorite_id, win_loss):
    if(bet_type == team_favorite_id): 
        W = win_dict[fav_spread]
    elif fav_spread == 0:
        W = .5
    else:
        W = 1 - win_dict[fav_spread]
    R = sum(win_loss)/len(win_loss)
    K = W - ((1 - W)/(R))
    return K/100

win_loss = []
for x in range(0,60): #30 moving average start w/ 66% track record 
    if x % 3 == 0: 
        win_loss.append(0)
    else:
        win_loss.append(1)
        
def simple_simulation(self, current_amount, plus_ev_df):
    current_amount = current_amount
    #bet_amount = 25 #bet amount is from kelley criteria = W - ((1 - W)/R)
    for i, row in plus_ev_df.iterrows():
        kelly_percent = self.kelly_criterion(win_dict, row.spread_favorite, row.Bet_Type, row.team_favorite_id, win_loss)
        if kelly_percent < 0:
            continue #go to next game
        bet_amount = current_amount * kelly_percent
        print(bet_amount)
        if row.Bet_Type == row.home:
            win = (bet_amount * (1 + row.implied_win_home)) - bet_amount
            lose = bet_amount
            if row.score_home > row.score_away:
                win_loss.append(1) 
                win_loss.pop(0)#increase w/l ratio
                start_amount = current_amount + win
            elif row.score_home < row.score_away:
                start_amount = current_amount - lose
                win_loss.append(0)
                win_loss.pop(0)#increase w/l ratio

                
        elif row.Bet_Type == row.away:
            win = (bet_amount * (1 + row.implied_win_away)) - bet_amount 
            lose = bet_amount
            if row.score_away > row.score_home:
                start_amount = current_amount + win
                win_loss.append(1)
                win_loss.pop(0)#increase w/l ratio
            elif row.score_away < row.score_home:
                start_amount = current_amount - lose
                win_loss.append(0)
                win_loss.pop(0)#increase w/l ratio

        print(current_amount)
    return current_amount


In [41]:
simple_simulation(ev_dates)



5.701349999999998
1003.81660341835
7.203378642463999
1008.961875935361
7.199514155655884
1014.6047399383376
6.362842449984007
1019.2123159443842
8.383299398003054
1010.8290165463811
8.213654632395906
1017.1969069234397
4.215610077335893
1019.8511814964327
3.791697423034291
1016.0594840733984
2.1052928461765124
1017.3821406198873
6.380260230782324
1022.0943583764544
3.800037314333831
1018.2943210621205
8.349820453517326
1024.7135461292983
8.40245688254675
1031.2893845076912
7.4417164281622314
1023.8476680795291
8.904631662120908
1014.9430364174082
1.9834778024946296
1012.9595586149136
3.578912740531317
1015.2129494437344
1.9840052867741533
1016.3937143181157
4.123890446631463
1019.0421509929705
2.111472983684927
1016.9306780092855
9.215934269459149
1024.2848092376285
2.0017341955728654
1022.2830750420555
4.7276247517626135
1017.5554502902929
4.621610834889667
1020.5025637090962
4.047262142542088
1016.4553015665541
3.4901361084501286
1018.6967297978507
4.040100295541785
1021.331578167995

1212.8050734215144

In [ ]:
#simulate from 2009-2011 w/ bayes updating after each game


In [8]:
df_historic_win_data = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "1990-9-10") #winners by spread since
win_dict = algorithms.win_percent(df_historic_win_data)
win_dict

{-1.0: 0.502762,
 -1.5: 0.505792,
 -2.0: 0.590747,
 -2.5: 0.578544,
 -3.0: 0.678171,
 -3.5: 0.741325,
 -4.0: 0.762295,
 -4.5: 0.784983,
 -5.0: 0.828054,
 -5.5: 0.837456,
 -6.0: 0.845946,
 -6.5: 0.848635,
 -7.0: 0.885362,
 -7.5: 0.926923,
 -8.0: 0.883721,
 -8.5: 0.918919,
 -9.0: 0.923913,
 -9.5: 0.9625,
 -10.0: 0.947115,
 -10.5: 0.969925,
 -11.0: 1.0,
 -11.5: 0.947368,
 -12.0: 0.956522,
 -12.5: 0.93617,
 -13.0: 0.975,
 -13.5: 0.968085,
 -14.0: 0.986486,
 -14.5: 0.977778,
 -15.0: 1.0,
 -15.5: 1.0,
 -16.0: 1.0,
 -16.5: 0.944444,
 -17.0: 1.0,
 -17.5: 1.0,
 -18.0: 1.0,
 -18.5: 1.0,
 -19.0: 1.0,
 -19.5: 1.0,
 -20.0: 1.0,
 -20.5: 1.0,
 -21.0: 1.0,
 -21.5: 1.0,
 -22.5: 1.0,
 -24.0: 1.0,
 -24.5: 1.0,
 -26.5: 1.0}

In [3]:
#start day default 2009-09-10
def historic_win_set(ey, em, ed, sy = 2009, sm = 9, sd = 10 ):
    start_date = datetime.date(sy,sm,sd)
    end_date = datetime.date(ey, em, ed)
    return start_date, end_date

#https://www.investopedia.com/terms/w/win-loss-ratio.asp
#w_l is a trader's number of winning trades relative to the number of osing trades.

#A negative Kelly criterion means that the bet is not favored by the model and should be avoided.

def kelly_criterion(win_dict, fav_spread, bet_type, team_favorite_id, win_loss):
    if(bet_type == team_favorite_id): 
        W = win_dict[fav_spread]
    elif fav_spread == 0:
        W = .5
    else:
        W = 1 - win_dict[fav_spread]
    R = sum(win_loss)/len(win_loss)
    K = W - ((1 - W)/(R))
    return K/100

win_loss = []
for x in range(0,60): #30 moving average start w/ 66% track record 
    if x % 3 == 0: 
        win_loss.append(0)
    else:
        win_loss.append(1)
#start amount is for starting amount
def simulate_v2(self, df_historic_win, df_ml, algorithms, start_amount): #more advanced simulation with win_data updating everyday
    start_date, end_date = self.historic_win_set(2020,9,30) #set start date and end date. 
    delta = datetime.timedelta(days=1)
    amount = start_amount
    while start_date <= end_date: #simulate for everyday between the start date and end date, might update to only gamedays 
        check_date = str(start_date)
        games_today = df_historic_win[df_historic_win['schedule_date'] == check_date]
        if len(games_today) == 0: #check to see if values in row else pass
            start_date += delta #iterate date
            continue
        else:
            win_data = df_historic_win[df_historic_win['schedule_date'] > check_date] #create win_df for days before the event
            win_dict = algorithms.win_percent(win_data) #update win percentage dict based on days before
            start_date += delta #iterate through date
            df_ml_today = df_ml[df_ml['schedule_date'] == check_date]
            plus_ev_df = algorithms.plus_ev_games(win_dict ,df_ml_today, .05 , largest_spread_fav=-15) # choose largest spread, ev. Should just be games for today
            amount += self.simple_simulation(self, amount, plus_ev_df)
    return amount
        

In [1]:
import datetime
import algorithms
import data_clean
import simulations
import pandas as pd
import numpy as np

In [2]:
df = data_clean.do(data_clean)
df = algorithms.ml_probabilities(df)
df_historic_win_data = data_clean.clean_data("spreadspoke_scores.csv", "nfl_teams.csv", "1990-9-10") #winners by spread since
#win_dict = algorithms.win_percent(df_historic_win_data)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [ ]:
start_date, end_date = simulations.historic_win_set(2019,11,15, 2015, 9, 10)
my_sim = simulations.Simulations(1000,0.02, -20, start_date, end_date) #spread is negative
my_sim.simulate_v2(df_historic_win_data,df,algorithms)


In [ ]:
my_sim.current_amount

In [ ]:
vars(my_sim)




#